In [ ]:
# pip install tensorflow==1.15.0

In [ ]:
import os
print(os.getcwd())

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  3 23:19:18 2019

@author: owgs (ghimsiong.ow@gmail.com)

"""

import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import pandas as pd
import re, string, time
import matplotlib.pyplot as plt
    
class HMM(object):
    '''
    This HMM class comprises functions from two sources:
    A) Nishant Shukla - Machine Learning with TensorFlow (2018, Manning Publications)
       o used for implementing the forward algorithm
       o used for implementing the Viterbi algorithm
    B) Marvin Bertin - https://github.com/MarvinBertin/HiddenMarkovModel_TensorFlow
       o used for implementing the Baum-Welch algorithm
    '''    
    
    def __init__(self, initial_prob, trans_prob, obs_prob,
                 epsilon=0.1, maxStep=5):
        
        T = trans_prob          # No need to convert as required format is the same.
        E = obs_prob.T          # To convert the format from Source (A) to Source (B)
        T0 = initial_prob.T[0]  # to convert to a row vector      
        
        with tf.name_scope('Inital_Parameters'):
            with tf.name_scope('Scalar_constants'):
                # Max number of iteration
                self.maxStep = maxStep

                # convergence criteria
                self.epsilon = epsilon 

                # Number of possible states
                self.S = T.shape[0]

                # Number of possible observations
                self.O = E.shape[0]
                
                self.prob_state_1 = []

            with tf.name_scope('Model_Parameters'):
                # Emission probability
                self.E = tf.Variable(E, dtype=tf.float64, name='emission_matrix')

                # Transition matrix
                self.T = tf.Variable(T, dtype=tf.float64, name='transition_matrix')

                # Initial state vector
                self.T0 = tf.Variable(tf.constant(T0, dtype=tf.float64, name='inital_state_vector'))
		
        

    def initialize_forw_back_variables(self, shape):
        self.forward = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='forward')
        self.backward = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='backward')
        self.posterior = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='posteriror')


    def _forward(self, obs_prob_list):
        
        with tf.name_scope('init_scaling_factor'):
            self.scale = tf.Variable(tf.zeros([self.N], tf.float64)) #scale factors
        
        with tf.name_scope('forward_first_step'):
            # initialize with state starting priors
            init_prob = tf.multiply(self.T0, tf.squeeze(obs_prob_list[0]))

            # scaling factor at t=0
            #owgs - self.scale = tf.scatter_update(self.scale, 0, 1.0 / tf.reduce_sum(init_prob))
            self.scale = tf.scatter_update(self.scale, 0, 1.0 / tf.reduce_sum(init_prob))
            
            # scaled belief at t=0
            #owgs self.forward = tf.scatter_update(self.forward, 0, self.scale[0] * init_prob)
            self.forward = tf.scatter_update(self.forward, 0, self.scale[0] * init_prob)
            
        # propagate belief
        for step, obs_prob in enumerate(obs_prob_list[1:]):
            with tf.name_scope('time_step-%s' %step):
                # previous state probability
                prev_prob = tf.expand_dims(self.forward[step, :], 0)
                # transition prior
                prior_prob = tf.matmul(prev_prob, self.T)
                # forward belief propagation
                forward_score = tf.multiply(prior_prob, tf.squeeze(obs_prob))

                forward_prob = tf.squeeze(forward_score)
                # scaling factor
                #owgs - self.scale = tf.scatter_update(self.scale, step+1, 1.0 / tf.reduce_sum(forward_prob))
                self.scale = tf.scatter_update(self.scale, step+1, 1.0 / tf.reduce_sum(forward_prob))
                
                # Update forward matrix
                #owgs - self.forward = tf.scatter_update(self.forward, step+1, self.scale[step+1] * forward_prob)
                self.forward = tf.scatter_update(self.forward, step+1, self.scale[step+1] * forward_prob)
        

    def _backward(self, obs_prob_list):
        with tf.name_scope('backward_last_step'):
            # initialize with state ending priors
            #owgs self.backward = tf.scatter_update(self.backward, 0, self.scale[self.N-1] * tf.ones([self.S], dtype=tf.float64)) 
            self.backward = tf.scatter_update(self.backward, 0, self.scale[self.N-1] * tf.ones([self.S], dtype=tf.float64)) 

        # propagate belief
        for step, obs_prob in enumerate(obs_prob_list[:-1]):
            with tf.name_scope('time_step-%s' %step):
                # next state probability
                next_prob = tf.expand_dims(self.backward[step, :], 1)
                # observation emission probabilities
                obs_prob_d = tf.linalg.tensor_diag(tf.squeeze(obs_prob)) #owgs - tf.diag(tf.squeeze(obs_prob))
                # transition prior
                prior_prob = tf.matmul(self.T, obs_prob_d)
                # backward belief propagation
                backward_score = tf.matmul(prior_prob, next_prob)

                backward_prob = tf.squeeze(backward_score)

                # Update backward matrix
                #owgs self.backward = tf.scatter_update(self.backward, step+1, self.scale[self.N-2-step] * backward_prob)
                self.backward = tf.scatter_update(self.backward, step+1, self.scale[self.N-2-step] * backward_prob)
        
        self.backward = tf.assign(self.backward, tf.reverse(self.backward, [True, False])) #owgs- tf.assign(self.backward, tf.reverse(self.backward, [True, False]))

        
    def _posterior(self):
        # posterior score
        self.posterior = tf.multiply(self.forward, self.backward)

        marginal = tf.reduce_sum(self.posterior, 1)
        self.posterior = self.posterior / tf.expand_dims(marginal, 1)       
        
        
    def re_estimate_emission(self, x):
        
        states_marginal = tf.reduce_sum(self.gamma, 0)
        seq_one_hot = tf.one_hot(tf.cast(x, tf.int64), self.O, 1, 0)
        emission_score = tf.matmul(tf.cast(seq_one_hot, tf.float64), self.gamma, transpose_a=True)
        return emission_score / states_marginal
    
    def re_estimate_transition(self, x):
        
        with tf.name_scope('Init_3D_tensor'):
            self.M = tf.Variable(tf.zeros((self.N-1, self.S, self.S), tf.float64))
        
        with tf.name_scope('3D_tensor_transition'):
            for t in range(self.N - 1):
                with tf.name_scope('time_step-%s' %t):
                    tmp_0 = tf.matmul(tf.expand_dims(self.forward[t, :], 0), self.T)
                    tmp_1 = tf.multiply(tmp_0, tf.expand_dims(tf.gather(self.E, x[t+1]), 0))
                    denom = tf.squeeze(tf.matmul(tmp_1, tf.expand_dims(self.backward[t+1, :], 1)))

                with tf.name_scope('Init_new_transition'):
                    trans_re_estimate = tf.Variable(tf.zeros((self.S, self.S), tf.float64))
                    
                for i in range(self.S):
                    with tf.name_scope('State-%s' %i):
                        numer = self.forward[t, i] * self.T[i, :] * tf.gather(self.E, x[t+1]) * self.backward[t+1, :]
                        #owgs trans_re_estimate = tf.scatter_update(trans_re_estimate, i, numer / denom)
                        trans_re_estimate = tf.scatter_update(trans_re_estimate, i, numer / denom)
                        
                #self.M = tf.scatter_update(self.M, t, trans_re_estimate)
                self.M = tf.scatter_update(self.M, t, trans_re_estimate)

        with tf.name_scope('Smooth_gamma'):
            self.gamma = tf.squeeze(tf.reduce_sum(self.M, 2))
            T_new = tf.reduce_sum(self.M, 0) / tf.expand_dims(tf.reduce_sum(self.gamma, 0), 1)
        
        with tf.name_scope('New_init_states_prob'):
            T0_new = self.gamma[0,:]

        with tf.name_scope('Append_gamma_final_time_step'):
            prod = tf.expand_dims(tf.multiply(self.forward[self.N-1, :], self.backward[self.N-1, :]), 0)
            s= prod/ tf.reduce_sum(prod)
            self.gamma = tf.concat([self.gamma, s], 0)
            
            self.prob_state_1.append(self.gamma[:, 0])
        
        return T0_new, T_new
    
    def check_convergence(self, new_T0, new_transition, new_emission):
        
        delta_T0 = tf.reduce_max(tf.abs(self.T0 - new_T0)) < self.epsilon
        delta_T = tf.reduce_max(tf.abs(self.T - new_transition)) < self.epsilon
        delta_E = tf.reduce_max(tf.abs(self.E - new_emission)) < self.epsilon

        return tf.logical_and(tf.logical_and(delta_T0, delta_T), delta_E)
        
    def forward_backward(self, obs_prob_seq):
        obs_prob_list_for = tf.split(obs_prob_seq, self.N, 0)
        
        with tf.name_scope('forward_belief_propagation'):
            # forward belief propagation
            self._forward(obs_prob_list_for)

        obs_prob_seq_rev = tf.reverse(obs_prob_seq, [True, False])
        obs_prob_list_back = tf.split(obs_prob_seq_rev, self.N, 0)

        with tf.name_scope('backward_belief_propagation'):
            # backward belief propagation
            self._backward(obs_prob_list_back)
        
    def expectation_maximization_step(self, x):
        
        # probability of emission sequence
        obs_prob_seq = tf.gather(self.E, x)

        with tf.name_scope('Forward_Backward'):
            self.forward_backward(obs_prob_seq)

        with tf.name_scope('Re_estimate_transition'):
            new_T0, new_transition = self.re_estimate_transition(x)
        
        with tf.name_scope('Re_estimate_emission'):
            new_emission = self.re_estimate_emission(x)

        with tf.name_scope('Check_Convergence'):
            converged = self.check_convergence(new_T0, new_transition, new_emission)

        with tf.name_scope('Update_parameters'):
            self.T0 = tf.assign(self.T0, new_T0)
            self.E = tf.assign(self.E, new_emission)
            self.T = tf.assign(self.T, new_transition)
            #self.count = tf.assign_add(self.count, 1)
             
            with tf.name_scope('histogram_summary'):
                _ = tf.summary.histogram(self.T0.name, self.T0) #owgs - tf.summary.histogram(self.T0.name, self.T0)
                _ = tf.summary.histogram(self.T.name, self.T) #owgs - tf.summary.histogram(self.T.name, self.T)
                _ = tf.summary.histogram(self.E.name, self.E) #owgs - tf.summary.histogram(self.E.name, self.E)
        return converged
        
    
    def Baum_Welch_EM(self, obs_seq):
        
        with tf.name_scope('Input_Observed_Sequence'):
            # length of observed sequence
            self.N = len(obs_seq)

            # shape of Variables
            shape = [self.N, self.S]

            # observed sequence
            x = tf.constant(obs_seq, dtype=tf.int32, name='observation_sequence')
        
        with tf.name_scope('Initialize_variables'):
            # initialize variables
            self.initialize_forw_back_variables(shape)
        
        converged = tf.cast(False, tf.bool)
        #self.count = tf.Variable(tf.constant(0))
        
        with tf.name_scope('Train_Baum_Welch'):
            for i in range(self.maxStep):
                
                with tf.name_scope('EM_step-%s' %i):
                    converged = self.expectation_maximization_step(x)
      
        return converged
              
def run_Baum_Welch_EM(sess, hmm, obs_seq, summary=False, monitor_state_1=False):
    
    converged = hmm.Baum_Welch_EM(obs_seq)
    
    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all() #owgs - tf.summary.merge_all()
    
    # Run 
    sess.run(tf.global_variables_initializer())
    trans0, transition, emission, c = sess.run([hmm.T0, hmm.T, hmm.E, converged])

    return trans0, transition, emission, c    


In [ ]:
from google.colab import files
fp = files.upload()

In [ ]:
# Initalise the session
sess = tf.Session()
     
# Load and prepare the data
fp = "text.txt"
s = open(fp).read()[:10000]
s2 = re.sub("[^a-z]", "_", s.lower())
    
# Prepare the dataframe of data
alpha_states = ["_"] + list(string.ascii_lowercase)
mapper = dict(zip(alpha_states, range(len(alpha_states))))
V = pd.Series(list(s2)).map(mapper).values
data = pd.DataFrame({"Visible": V, "Alphabet": list(s2)})
data["Hidden"] = data["Alphabet"].map({"a": 1, "e": 1, "i": 1, "o": 1, "u": 1}).replace(np.nan, 0).astype(int)
data = data[["Alphabet", "Hidden", "Visible"]]    

In [ ]:
# Define the initial distributions
initial_distribution = np.array([[0.6],[0.4]])
initial_T = np.array([[0.6177499,0.3822501], [0.8826096,0.1173904]])
initial_E = np.array([
            [0.037192964,0.009902360,0.032833978,0.044882670,0.057331132,
             0.052143890,0.013665015,0.036187536,0.072293323,0.044793972,0.060008388,
             0.004256270,0.024770706,0.053520546,0.014232306,0.046981769,0.053733382,
             0.066355203,0.046817817,0.006912535,0.016201697,0.013425499,0.024694447,
             0.064902148,0.046170421,0.033586536,0.022203489],
            [0.0389931197,0.0697183142,0.0239154174,0.0512772632,0.0404732634,0.0059687348,
             0.0211687193,0.0625229746,0.0039632091,0.0567828864,0.0468108656,0.0168355418,
             0.0627882213,0.0286478204,0.0389215263,0.0064318198,0.0001698078,0.0493758725,
             0.0652709152,0.0069580806,0.0093043072,0.0028807932,0.0521827110,0.0608822385,
             0.0645417465,0.0555249876,0.0576888424]
            ])

In [ ]:
num_chars = 100
maxStep = 2

# Start logs
print ("Start time: %s" % time.asctime())
start_time = time.perf_counter()
            
# Initialise the model
hmm =  HMM(initial_distribution, initial_T, initial_E,epsilon=0.1, maxStep=maxStep)
            
# Run the baum welch algorithm
trans0, transition, emission, c = run_Baum_Welch_EM(
                sess, hmm,
                data["Visible"].values[:num_chars], summary=False, monitor_state_1=True)       
        
# End logs
print ("End time: %s" % time.asctime())
end_time = time.perf_counter()
print ("Time taken: %s hrs" % ((end_time-start_time)/60/60))
        
print("Transition Matrix: ")
print(transition)
print()
print("Emission Matrix: ")
print(emission)
print()
print("Reached Convergence: ")
print(c) 
    
# plot
plt.bar(alpha_states, emission[:,0])